In [1]:
import matplotlib.pyplot as plt
import uproot
import numpy as np
import pandas as pd

from pathlib import Path
from sklearn.model_selection import train_test_split

from datetime import datetime
import sys
sys.path.insert(1, '/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/util_funcs/')
from pandas_colFuncs import isBtoDstlnu, whichBisSig, customMCmatching, B_ID

In [14]:
root_path_finalDF = "/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_CMvars_run1/"

In [15]:
#df_FSPs = pd.read_csv(root_path_finalDF + "final_df_FSPs_preProcessed_SHR_wLabels.csv")
#df_Y4S = pd.read_csv(root_path_finalDF + "final_df_Y4S_preProcessed_SHR.csv")


In [2]:
nfs_path = "/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/"

data_subdir = "FULLCMrun_Dstlnu_SHR_BsX_Ablations/"
root_subdir = "SHR_CMvars_run1/"   # "SHR_LeasVars_run2/"

root_path = nfs_path + "SHR_Hc_correctReco_BsX/" + root_subdir # + sub + "/"
merged = "merged_"

In [3]:
names = ["gammas","electrons","pions","kaons","muons"]
dfs = []
for name in names:
    filename = root_path + merged + "{}.root:variables".format(name)
    #filename = afsPath + "{}.root:variables".format(name)
    print(filename)
    tmpFileFSPs = uproot.open(filename)
    df_tmp = tmpFileFSPs.arrays(library="pd")
    dfs.append(df_tmp)

/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_CMvars_run1/merged_gammas.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_CMvars_run1/merged_electrons.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_CMvars_run1/merged_pions.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_CMvars_run1/merged_kaons.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_CMvars_run1/merged_muons.root:variables


In [4]:
df_FSPs = pd.concat(dfs)

In [5]:
df_FSPs.shape[0]

9079729

In [6]:
fileY4S = uproot.open(root_path + merged + "DXtagDstl.root:variables")
df_Y4S = fileY4S.arrays(library="pd")

In [ ]:
df_Y4S = df_Y4S[(df_Y4S['Hc_isSignalAcceptMissingGamma'] == 1.0)]
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Y4S["__event__"])]

## add cols

In [8]:
# function to create col with the particles mother B's uniqueParticleIdentifier
df_FSPs['B_ID'] = df_FSPs.apply(B_ID, axis=1)

In [9]:
df_Y4S['Bsig_uniqParID'] = df_Y4S.apply(whichBisSig, axis=1)

In [10]:
def labels(s):
    label = -1
    if int(s['B_ID']) == 0:
        label = 0 # background, cause not related to MC Particles
    else: 
        event_num = (s['__event__'])
        B_sigID = df_Y4S[df_Y4S['__event__'] == event_num].iloc[0]['Bsig_uniqParID']
        
        if int(s['B_ID']) == B_sigID:
            label = 2 # Bsig
        else:
            label = 1 #X
    return label

In [11]:
df_FSPs['label'] = df_FSPs.apply(labels, axis=1)

IndexError: single positional indexer is out-of-bounds

## save df's with extra cols

In [ ]:
df_FSPs.to_csv(root_path + "df_FSPs_forAnalysis.csv")
df_Y4S.to_csv(root_path + "df_Y4S_forAnalysis.csv")

## load preprocessed

In [ ]:
df_FSPs = pd.read_csv(root_path + "df_FSPs_forAnalysis.csv")
df_Y4S = pd.read_csv(root_path + "df_Y4S_forAnalysis.csv")

## investigate which particle hypothesis to use

In [ ]:
groupsFSPs_uniqParID = pd.DataFrame({'count' : df_FSPs.groupby( ["__event__","uniqueParticleIdentifier"] ).size()}).reset_index()

In [ ]:
groupsFSPs_uniqParID.sort_values("count",ascending=False)

In [ ]:
df_FSPs[(df_FSPs["__event__"] == ) & (df_FSPs["uniqueParticleIdentifier"] == )][["uniqueParticleIdentifier","PDG","mcPDG","basf2_used","label"]]

## investigate cut on X gammas

In [17]:
df_FSPs[df_FSPs["basf2_X"] == 1.].shape[0]

385704

In [19]:
basf2Xgamma_df = df_FSPs[(df_FSPs["basf2_X"] == 1.) & (df_FSPs["PDG"] == 22.)]

In [20]:
basf2Xgamma_df[["clusterReg","E","label"]]

,clusterReg,E,label
37787,2.0,1.410958,1
37789,2.0,0.431910,1
37790,2.0,0.157982,1
37791,2.0,0.392263,2
37792,2.0,0.200342,1
...,...,...,...
707710,2.0,0.065446,1
707712,1.0,0.137836,1
707714,2.0,0.209976,1
707715,2.0,0.120469,2


In [21]:
basf2Xgamma_df.shape[0]

299883

In [22]:
#      goodGammaRegion1 = region == 1 && energy > 0.100;
#      goodGammaRegion2 = region == 2 && energy > 0.050;
#      goodGammaRegion3 = region == 3 && energy > 0.150;


In [24]:
goodGamma = basf2Xgamma_df[((basf2Xgamma_df["clusterReg"] == 1) & (basf2Xgamma_df["E"] >= 0.100)) |
         ((basf2Xgamma_df["clusterReg"] == 2) & (basf2Xgamma_df["E"] >= 0.050)) |
         ((basf2Xgamma_df["clusterReg"] == 3) & (basf2Xgamma_df["E"] >= 0.150))]

In [25]:
goodGamma.shape[0]

299883

In [1]:
for i in range(0,2):
    for j in range(0,3):
        print(f'genUp4S_PDG_{i}_{j}')
        print(f'abs_genUp4S_PDG_{i}_{j}')

genUp4S_PDG_0_0
abs_genUp4S_PDG_0_0
genUp4S_PDG_0_1
abs_genUp4S_PDG_0_1
genUp4S_PDG_0_2
abs_genUp4S_PDG_0_2
genUp4S_PDG_1_0
abs_genUp4S_PDG_1_0
genUp4S_PDG_1_1
abs_genUp4S_PDG_1_1
genUp4S_PDG_1_2
abs_genUp4S_PDG_1_2
